# Databases

The problems in this notebook expand upon the concepts covered in the notebook Lecture 2:  Data in Databases.

##### 1. Constructing a database table 

To better understand the structure of a relational database it can help to create one yourself. In this problem we demonstrate how.

Imagine we are running a cat store, we sell 3 products: cat food, cat treats, and cat toys. We want a database to keep track of who is buying what stuff from us.

Below we import the package we will need and then we create a `cat_store_practice` database in this folder.

In [1]:
from sqlalchemy import create_engine

In [2]:
## making the engine
engine = create_engine("sqlite:///cat_store_practice.db")

In [3]:
## Connecting to the database
conn = engine.connect()

The syntax for creating a brand new table in `SQL` is `CREATE TABLE table_name(columns)`. We demonstrate this below.

In [4]:
## CREATE TABLE is SQL code
## it creates a table with the given name, here products
## in parantheses we list the columns of our table
## along with the SQL data type
## The PRIMARY KEY line sets the product_id as the
## primary key for this table
## Think of a primary key as being equivalent to a pandas dataframe index
## The primary key allows us to link entries across tables
conn.execute("""CREATE TABLE products(
                    product_id int,
                    product text,
                    price real,
                    in_stock int,
                    PRIMARY KEY (product_id)
                )""")


## We can now add our first product
## with an INSERT command
## Here you write INSERT INTO table_name VALUES (values)
conn.execute("INSERT INTO products VALUES (1,'Cat Food',12.50,10)")

Use one of the `fetch` commands to check the contents of the `products` table.

In [5]:
conn.execute("SELECT * FROM products").fetchall()

[(1, 'Cat Food', 12.5, 10)]

In [6]:
## close the connection
conn.close()

## dispose of the engine
engine.dispose()

del conn,engine

##### 2. `inspect`

You can use `get_table_names` to see what tables are in the database to which you have connected. This is a part of `SQLAlchemy`'s `inspect` method, <a href="https://docs.sqlalchemy.org/en/14/core/inspection.html">https://docs.sqlalchemy.org/en/14/core/inspection.html</a>.

Here we demonstrate.

In [7]:
## import inspect
from sqlalchemy import inspect

In [8]:
## create the engine then connect
engine = create_engine("sqlite:///cat_store_real.db")
conn = engine.connect()

In [9]:
## inspecting the table allows you to
## use get_table_names
inspect(engine).get_table_names()

['customers', 'purchases']

<b>Do not</b> end the connection or delete the engine, you will use in in the next problem.

##### 3. Using `WHERE`

Create a connection to the `cat_store_real` database in this folder.

Return all purchases in the `purchases` table with `pretax_price < 70` and `number_of_items > 3`.

##### Sample Solution

In [10]:
conn.execute("SELECT * FROM purchases WHERE (pretax_price < 70) AND (number_of_items > 3)").fetchall()

[(1, 3, 4, 18.9, 'credit'),
 (7, 5, 4, 39.9, 'credit'),
 (11, 3, 5, 26.5, 'check'),
 (12, 2, 6, 12.54, 'debit'),
 (19, 9, 5, 68.23, 'debit')]

##### 4. Introduction to `JOIN`s

One way to combine data from different tables is with a `JOIN` statement.

`JOIN`s work by finding column values that match between tables and returning the corresponding rows. Here is a typical `JOIN` statement:

<blockquote>
    
    SELECT columns FROM table1
    
    JOIN table2
    
    ON table1.match_column=table2.match_column
    
    WHERE logical_condition;    
</blockquote>

Here is an example where we add the customer names to each purchase.

In [11]:
import pandas as pd

In [12]:
results = conn.execute("""SELECT name, purchase_id, pretax_price FROM purchases
                            JOIN customers
                            ON purchases.customer_id=customers.customer_id""")

pd.DataFrame(results.fetchall(), columns=results.keys())

,name,purchase_id,pretax_price
0,Melanie PBody,1,18.90
1,Francine Frensky,2,22.20
2,Olivia Olive,3,7.89
3,Mike Evans,4,109.89
4,Mark Ruffalo,5,33.30
5,Paul London,6,10.99
6,Frances Paris,8,71.89
7,Richard Frank,9,209.89
8,Mark Ruffalo,10,17.54
9,Melanie PBody,11,26.50


We should note that if the two tables you want to join share a column name you need to specify which one you want.

In [13]:
results = conn.execute("""SELECT name, purchases.customer_id, purchase_id, pretax_price FROM purchases
                            JOIN customers
                            ON purchases.customer_id=customers.customer_id""")

pd.DataFrame(results.fetchall(), columns=results.keys())

,name,customer_id,purchase_id,pretax_price
0,Melanie PBody,3,1,18.90
1,Francine Frensky,2,2,22.20
2,Olivia Olive,7,3,7.89
3,Mike Evans,1,4,109.89
4,Mark Ruffalo,4,5,33.30
5,Paul London,9,6,10.99
6,Frances Paris,8,8,71.89
7,Richard Frank,6,9,209.89
8,Mark Ruffalo,4,10,17.54
9,Melanie PBody,3,11,26.50


##### Practice

Try to answer the following using `JOIN` statements.

1. Who has made purchases?

2. Who has made the most purchases?

3. What are the emails of customers that have made purchases over $70?

##### Sample Solution

In [14]:
##### 1.

results = conn.execute("""SELECT name FROM purchases
                            JOIN customers
                            ON purchases.customer_id=customers.customer_id""")

pd.DataFrame(results).value_counts()

name            
Francine Frensky    3
Mark Ruffalo        3
Melanie PBody       3
Frances Paris       2
Mike Evans          2
Paul London         2
Jenny Gump          1
Olivia Olive        1
Richard Frank       1
dtype: int64

In [15]:
##### 2.

results = conn.execute("""SELECT name FROM purchases
                            JOIN customers
                            ON purchases.customer_id=customers.customer_id""")

df = pd.DataFrame(results.fetchall(), 
                     columns=results.keys())

df.name.value_counts().index[df.name.value_counts()==df.name.value_counts().max()]

Index(['Melanie PBody', 'Francine Frensky', 'Mark Ruffalo'], dtype='object')

In [16]:
##### 3.

results = conn.execute("""SELECT email FROM purchases
                            JOIN customers
                            ON purchases.customer_id=customers.customer_id
                            WHERE pretax_price > 70""")

pd.DataFrame(results.fetchall(),
                columns=results.keys())

,email
0,mik.evans@yahoo.com
1,iseelondon@gmail.com
2,letsbefrank@hotmail.com
3,mik.evans@yahoo.com
4,arthurfan@gmail.com


--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)